# BASE

In [1]:
# Importar paquetes que serán usados
import numpy as np
import pandas as pd
from sklearn import linear_model, datasets

from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from collections import Counter
from warnings import filterwarnings
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import imblearn
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

# Definicion del color para las gráficas
paleta=sns.color_palette("Blues_r")

# Lectura del conjunto de datos
conjunto_datos = pd.read_csv("/content/drive/MyDrive/conjunto_1.csv", sep=";", decimal=",")

# Se renombran las columnas para que sean más descriptivas
conjunto_datos.columns = ["FS365", "FS450", "LDF35","LDF42","Clases"]


"""
Función para la creacion del conjunto usando la técnica de sobremuestreo
Parametros:   X_trains: matriz con el conjunto de entrenamiento
              y_trains: vector con las clases objetivo
Retorna: El conjunto con las clases balanceadas usando la técnica de sobremuestreo
"""
def genera_sobremuestreo(X_trains,y_trains):
  conjunto_entrenamiento = pd.concat([X_trains, y_trains], axis=1) 
  cuenta_clases = conjunto_entrenamiento.Clases.value_counts()

  cant_clase_0 = cuenta_clases[0]
  cant_clase_1 = cuenta_clases[1]
  cant_clase_2 = cuenta_clases[2]

  conjunto_clase_0 = conjunto_entrenamiento[conjunto_entrenamiento['Clases'] == 0]
  conjunto_clase_1 = conjunto_entrenamiento[conjunto_entrenamiento['Clases'] == 1]
  conjunto_clase_2 = conjunto_entrenamiento[conjunto_entrenamiento['Clases'] == 2]

  conjunto_clase_2_sm = conjunto_clase_2.sample(cant_clase_1, replace=True)
  conjunto_clase_0_sm = conjunto_clase_0.sample(cant_clase_1, replace=True)

  conjunto_sm = pd.concat([conjunto_clase_1, conjunto_clase_2_sm, conjunto_clase_0_sm], axis=0)
  conjunto_sm.groupby('Clases').size()
  return conjunto_sm


"""
Función para la creacion del conjunto usando la técnica de SMOTE
Parametros:   X_trainm: matriz con el conjunto de entrenamiento
              y_trainm: vector con las clases objetivo
Retorna: El conjunto con las clases balanceadas usando la técnica de SMOTE
"""
def genera_smote(X_trainm,y_trainm):
  sobre_muestreo_smote = SMOTE()
  X, y = sobre_muestreo_smote.fit_resample(X_trainm, y_trainm)

  conjunto_smo = pd.concat([X, y], axis=1)
  conjunto_smo.groupby('Clases').size()
  return conjunto_smo


"""
Función que realiza el entrenamiento con el conjunto original creado para tal fin
Parametros:   algoritmo: es el modelo o algoritmos que se va a entrenar
              parametros: hiperparámetros del modelo
              X_traino, y_traino: conjunto de entrenamiento
Salida por pantalla 1: los mejores hiperparámetros posterior al entrenamiento con GridSearchCV
Salida por pantalla 2: puntuación generada por GridSearchCV
"""
def entrena_original(algoritmo, parametros, X_traino, y_traino):
  print("\n\n  Conjunto Original")
  gs = GridSearchCV(algoritmo, parametros, cv=3, n_jobs=-1, verbose=True)
  mejores_parametros = gs.fit(X_traino, y_traino)
  print(mejores_parametros.best_estimator_)
  print(mejores_parametros.best_score_)

"""
Función que realiza el entrenamiento con el conjunto de Sobremuestreo
Parametros:   algoritmo: es el modelo o algoritmos que se va a entrenar
              parametros: hiperparámetros del modelo
              conjunto_sm: es el conjunto de entrenamiento, con las clases balanceadas 
                                      usando la técnica de sobremuestreo
Salida por pantalla 1: los mejores hiperparámetros posterior al entrenamiento con GridSearchCV
Salida por pantalla 2: puntuación generada por GridSearchCV
"""
def entrena_sobremuestreo(algoritmo, parametros, conjunto_sm):
  print("\n\n  Conjunto Sobremuestreo")
  X_train_csm = conjunto_sm.drop('Clases', axis = 'columns')
  y_train_csm = conjunto_sm['Clases']
  gs = GridSearchCV(algoritmo, parametros, cv=3, n_jobs=-1, verbose=True)
  mejores_parametros = gs.fit(X_train_csm, y_train_csm)
  print(mejores_parametros.best_estimator_)
  print(mejores_parametros.best_score_)

"""
Función que realiza el entrenamiento con el conjunto de SMOTE
Parametros:   algoritmo: es el modelo o algoritmos que se va a entrenar
              parametros: hiperparámetros del modelo
              conjunto_smo: es el conjunto de entrenamiento, con las clases balanceadas 
                                      usando la técnica de SMOTE
Salida por pantalla 1: los mejores hiperparámetros posterior al entrenamiento con GridSearchCV
Salida por pantalla 2: puntuación generada por GridSearchCV
"""
def entrena_smote(algoritmo, parametros, conjunto_smo): 
  print("\n\n  Conjunto SMOTE ")
  X_train_smo = conjunto_smo.drop('Clases', axis = 'columns')
  y_train_smo = conjunto_smo['Clases']
  gs = GridSearchCV(algoritmo, parametros, cv=3, n_jobs=-1, verbose=True)
  mejores_parametros = gs.fit(X_train_smo, y_train_smo)
  print(mejores_parametros.best_estimator_)
  print(mejores_parametros.best_score_)


"""
Función que realiza el reentrenamiento del modelo, lo evalúa y genera métricas
Parametros:   conjunto_entrenamiento, X_test_x, y_test_x: conjuntos de entrenamiento y pruebas
              algoritmo:  es el modelo que se va a entrenar, ya tiene los hiperparámetros la instanciar el clasificador
              titulo:  hace referencia al tipo de set de entrenamiento, puede ser "Original", "Sobremuestreo" o "SMOTE"
Salida por pantalla 1: métrica de precision
Salida por pantalla 2: métrica de exactitud
Salida por pantalla 3: informe con las principales métricas de clasificación.
Salida por pantalla 4: matriz de confusión
"""
def genera_graficas(conjunto_entrenamiento, X_test_x, y_test_x, algoritmo, titulo):

  X_train_x = conjunto_entrenamiento.drop('Clases', axis = 'columns')
  y_train_x = conjunto_entrenamiento['Clases']

  algoritmo.fit(X_train_x, y_train_x)
  y_pred = algoritmo.predict(X_test_x)

  print("Precision: ",precision_score(y_test_x, y_pred, average='weighted',zero_division=False))
  print("Exactitud: ",accuracy_score(y_test_x, y_pred))

  print(classification_report(y_test_x, y_pred,zero_division=False))
  matriz_confusion = confusion_matrix(y_test_x, y_pred)

  plt.figure(figsize=(7,7))
  sns.heatmap(matriz_confusion, annot=True, cmap=paleta, linecolor='white', linewidths=0.1, square=True)
  plt.title(titulo,size=16)
  plt.xlabel('Etiqueta Predicha',size=14)
  plt.ylabel('Etiqueta Verdadera',size=14)
  plt.show()


# División del conjunto de datos, en subconjuntos de entrenamiento y pruebas
X_train, X_test, y_train, y_test = train_test_split(conjunto_datos.drop('Clases', axis = 'columns'), 
    conjunto_datos['Clases'],train_size= 0.7,random_state = 42,shuffle= True)

# Une la matriz y vector de entrenamiento en un mismo conjunto
conjunto_original = pd.concat([X_train, y_train], axis=1)

# Generación de los conjuntos de entrenamiento, con las clases balanceadas usando _
# las técnicas de Sobremuestreo y SMOTE
conjunto_sobremuestreo = genera_sobremuestreo(X_train,y_train) 
conjunto_smote = genera_smote(X_train, y_train)



## Entrenamiento

### Regresión Logística

In [ ]:

# Inicio el clasificador Regesion Logística
clasificador_reglog = LogisticRegression(random_state=42)  

# Se establecen los hiperparámetros del clasificador Regesion Logística
hiper_parametros = {}                                     
hiper_parametros['classifier__penalty'] = ['l1', 'l2']
hiper_parametros['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
hiper_parametros['classifier__solver'] = ['liblinear','lbfgs','newton-cg','sag','saga']
hiper_parametros['classifier'] = [clasificador_reglog]
pipeline = Pipeline([('classifier', clasificador_reglog)])
filterwarnings('ignore')

# Se realiza entrenamiento del modelo usando GridSearchCV
entrena_original(pipeline, hiper_parametros, X_train, y_train)  
entrena_sobremuestreo(pipeline, hiper_parametros, conjunto_sobremuestreo)  
entrena_smote(pipeline, hiper_parametros, conjunto_smote)  


### KNN

In [ ]:
# Inicio del clasificador K-Vecinos más Cercanos 
clasificador_veccer = KNeighborsClassifier()                

# Se establecen los hiperparámetros del clasificador K-Vecinos más Cercanos   
hiper_parametros = {}                                     
hiper_parametros['classifier__n_neighbors'] = [5,10,20,50]
hiper_parametros['classifier__weights'] = ['uniform', 'distance']
hiper_parametros['classifier__leaf_size'] = [20,40,1]
hiper_parametros['classifier__p'] = [1,2]
hiper_parametros['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
hiper_parametros['classifier__metric'] = ['minkowski', 'chebyshev']
hiper_parametros['classifier'] = [clasificador_veccer]   
pipeline = Pipeline([('classifier', clasificador_veccer)])
filterwarnings('ignore')

# Se realiza entrenamiento del modelo usando GridSearchCV
entrena_original(pipeline, hiper_parametros, X_train, y_train)  
entrena_sobremuestreo(pipeline, hiper_parametros, conjunto_sobremuestreo)  
entrena_smote(pipeline, hiper_parametros, conjunto_smote)  

### SVMs

In [2]:
# Inicio del clasificador Máquina de Soporte de Vectores
clasificador_maqsop = SVC(random_state=42)              

# Se establecen los hiperparámetros del clasificador Máquina de Soporte de Vectores
hiper_parametros = {}                                      
hiper_parametros['classifier__C'] = [0.1,1, 10, 100] 
hiper_parametros['classifier__gamma'] = [1,0.1,0.01,0.001]
hiper_parametros['classifier__kernel'] = ['rbf', 'poly', 'sigmoid']
hiper_parametros['classifier__degree'] = [0, 1, 2, 3, 4, 5, 6]
hiper_parametros['classifier'] = [clasificador_maqsop]   
pipeline = Pipeline([('classifier', clasificador_maqsop)])
filterwarnings('ignore')

# Se realiza entrenamiento del modelo usando GridSearchCV
entrena_original(pipeline, hiper_parametros, X_train, y_train)  
entrena_sobremuestreo(pipeline, hiper_parametros, conjunto_sobremuestreo)  
entrena_smote(pipeline, hiper_parametros, conjunto_smote)  



  Conjunto Original
Fitting 3 folds for each of 336 candidates, totalling 1008 fits
Pipeline(steps=[('classifier',
                 SVC(C=10, degree=0, gamma=0.1, random_state=42))])
0.8025451559934318


  Conjunto Sobremuestreo
Fitting 3 folds for each of 336 candidates, totalling 1008 fits


KeyboardInterrupt: ignored

### Naive Bayes

In [4]:
# Inicio del clasificador Naive Bayes
clasificador_naibay = GaussianNB()                        

# Se establecen los hiperparámetros del clasificador Naive Bayes
hiper_parametros = {}                                      
hiper_parametros['classifier__var_smoothing'] = np.logspace(0,-9, num=100)
hiper_parametros['classifier'] = [clasificador_naibay]
pipeline = Pipeline([('classifier', clasificador_naibay)])
filterwarnings('ignore')

# Se realiza entrenamiento del modelo usando GridSearchCV
entrena_original(pipeline, hiper_parametros, X_train, y_train)  
entrena_sobremuestreo(pipeline, hiper_parametros, conjunto_sobremuestreo)  
entrena_smote(pipeline, hiper_parametros, conjunto_smote)  



  Conjunto Original
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Pipeline(steps=[('classifier',
                 GaussianNB(var_smoothing=0.0012328467394420659))])
0.8136288998357964


  Conjunto Sobremuestreo
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Pipeline(steps=[('classifier',
                 GaussianNB(var_smoothing=0.0015199110829529332))])
0.8652482269503546


  Conjunto SMOTE 
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Pipeline(steps=[('classifier',
                 GaussianNB(var_smoothing=0.0003511191734215131))])
0.8723404255319149


### Redes Neuronales

In [ ]:
# Inicio del clasificador Redes Neuronales
clasificador_redneu = MLPClassifier(random_state=42)  

# Se establecen los hiperparámetros del clasificador Redes Neuronales
hiper_parametros = {}                                     
hiper_parametros['classifier__alpha'] = [0.1, 0.01, 0.001]
hiper_parametros['classifier__max_iter'] = [5,10,20]
hiper_parametros['classifier__batch_size'] = [2,5,20]
hiper_parametros['classifier__activation'] = ['identity', 'logistic', 'tanh', 'relu']
hiper_parametros['classifier__hidden_layer_sizes'] = [1,3,5,10,50]  
hiper_parametros['classifier'] = [clasificador_redneu]
pipeline = Pipeline([('classifier', clasificador_redneu)])
filterwarnings('ignore')

# Se realiza entrenamiento del modelo usando GridSearchCV
entrena_original(pipeline, hiper_parametros, X_train, y_train)  
entrena_sobremuestreo(pipeline, hiper_parametros, conjunto_sobremuestreo)  
entrena_smote(pipeline, hiper_parametros, conjunto_smote)   

## Reentrenamiento y evaluación de modelos

El código que se encuentra en la parte inferior, cuenta con los hiperparámetros que mejores resultados proporcionaron al realizar el entrenamiento con GridSearchCV. Es posible que haya una variación, y se debe revisar las salidas por pantalla de la sección "Entrenamiento".

### Regresión Logística

In [ ]:
print("\n************ Original ************\n")
clasificador = LogisticRegression(C=10, penalty='l1', random_state=42,solver='liblinear')
genera_graficas(conjunto_original, X_test, y_test, clasificador, "Original") 

print("\n************ Sobremuestreo ************\n")
clasificador = LogisticRegression(C=1, random_state=42)
genera_graficas(conjunto_sobremuestreo, X_test, y_test, clasificador,"Sobremuestreo") 

print("\n************ SMOTE ************\n")
clasificador = LogisticRegression(C=10, random_state=42)
genera_graficas(conjunto_smote, X_test, y_test, clasificador, "SMOTE") 

### KNN

In [ ]:
print("\n************ Original ************\n")
clasificador = KNeighborsClassifier(leaf_size=20, n_neighbors=10, p=1)
genera_graficas(conjunto_original, X_test, y_test, clasificador, "Original") 

print("\n************ Sobremuestreo ************\n")
clasificador = KNeighborsClassifier(leaf_size=20, n_neighbors=10, p=1, weights='distance')
genera_graficas(conjunto_sobremuestreo, X_test, y_test, clasificador,"Sobremuestreo") 

print("\n************ SMOTE ************\n")
clasificador = KNeighborsClassifier(leaf_size=20, n_neighbors=10, p=1, weights='distance')
genera_graficas(conjunto_smote, X_test, y_test, clasificador, "SMOTE") 

### SVMs

In [ ]:
print("\n************ Original ************\n")
clasificador = SVC(C=10, degree=0, gamma=0.1, random_state=42)
genera_graficas(conjunto_original, X_test, y_test, clasificador, "Original") 

print("\n************ Sobremuestreo ************\n")
clasificador = SVC(C=0.1, degree=6, gamma=1, random_state=42)
genera_graficas(conjunto_sobremuestreo, X_test, y_test, clasificador,"Sobremuestreo") 

print("\n************ SMOTE ************\n")
clasificador = SVC(C=1, degree=0, gamma=0.1, random_state=42)
genera_graficas(conjunto_smote, X_test, y_test, clasificador, "SMOTE") 

### Naive Bayes

In [ ]:
print("\n************ Original ************\n")
clasificador =  GaussianNB(var_smoothing=0.0012328467394420659)
genera_graficas(conjunto_original, X_test, y_test, clasificador, "Original") 

print("\n************ Sobremuestreo ************\n")
clasificador = GaussianNB(var_smoothing=0.0015199110829529332)
genera_graficas(conjunto_sobremuestreo, X_test, y_test, clasificador,"Sobremuestreo") 

print("\n************ SMOTE ************\n")
clasificador = GaussianNB(var_smoothing=0.0003511191734215131)
genera_graficas(conjunto_smote, X_test, y_test, clasificador, "SMOTE") 

### Redes Neuronales

In [ ]:
print("\n************ Original ************\n")
clasificador = MLPClassifier(activation='tanh', alpha=0.1, batch_size=2,
                               hidden_layer_sizes=10, max_iter=20,
                               random_state=42)
genera_graficas(conjunto_original, X_test, y_test, clasificador, "Original") 

print("\n************ Sobremuestreo ************\n")
clasificador = MLPClassifier(activation='identity', alpha=0.01, batch_size=2,
                               hidden_layer_sizes=50, max_iter=10,
                               random_state=42)
genera_graficas(conjunto_sobremuestreo, X_test, y_test, clasificador,"Sobremuestreo") 

print("\n************ SMOTE ************\n")
clasificador = MLPClassifier(activation='logistic', alpha=0.01, batch_size=2,
                               hidden_layer_sizes=50, max_iter=20,
                               random_state=42)
genera_graficas(conjunto_smote, X_test, y_test, clasificador, "SMOTE") 